In [72]:
import csv
import networkx as nx
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from cdlib import algorithms
import igraph as ig
from netgraph import Graph
import networkx.algorithms.community as nxcom
import random
# Function to read node data from CSV file
def draw_communities(graph, communities):
    pos = nx.spring_layout(graph)  # You can use other layouts as well
    
    # Draw nodes and edges
    nx.draw(graph, pos, with_labels=True, node_color='lightgray', edge_color='gray', font_size=8, node_size=300)
    
    # Iterate over communities
    for idx, community in enumerate(communities):
        nx.draw_networkx_nodes(graph, pos, nodelist=community, node_color=plt.cm.tab10.colors[idx], node_size=300)
    
    # Show the plot
    plt.title("Graph with Communities")
    plt.show()
def read_nodes(filename):
    nodes = {}
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            nodes[int(row[0])] = int(row[1])
    return nodes

# Function to read edge data from CSV file
def read_edges(filename):
    edges = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            source, target, timestep = map(int, row)
            edges.append((source, target, timestep))
    return edges

# Function to create a graph for a specific time step
# def create_graph(timestep, nodes, edges):
#     G = nx.DiGraph()
#     for node, value in nodes.items():
#         if value <= timestep:  # Add nodes based on the timestep
#             G.add_node(node, value=value)
#     for edge in edges:
#         if edge[2] <= timestep:
#             G.add_edge(edge[0], edge[1])
#     # giant_component_size = len(max(nx.weakly_connected_components(G), key=len))
#     return G
def create_graph(timestep, nodes, edges):
    G = nx.Graph()
    
    # Randomly sample nodes based on the sampling ratio
   
    for node, value in nodes.items():
        if value <= timestep:  # Add nodes based on the timestep
            G.add_node(node, value=value)
    
    for edge in edges:
        if edge[2] <= timestep:
            # Check if both nodes of the edge are present in the sampled nodes
            if edge[0] in G.nodes() and edge[1] in G.nodes():
                G.add_edge(edge[0], edge[1])
    
    # giant_component_size = len(max(nx.weakly_connected_components(G), key=len))
    return G


In [73]:
nodes = read_nodes('final_nodes_time.csv')
edges = read_edges('final_edges_time.csv')
G = create_graph(40, nodes, edges)
print(nx.info(G))


Graph with 7337 nodes and 19480 edges


In [74]:
#TO ADD NEW NODES TO THE GRAPH:
for node,value in nodes.items():
    if value > 40 and value <= 43:
        G.add_node(node, value=value)

In [75]:
print(nx.info(G))   
lol = G.nodes()

Graph with 8145 nodes and 19480 edges


In [76]:
negetive_edges = []
for u in lol:
    for v in lol:
        if u != v and not G.has_edge(u,v):
            negetive_edges.append((u,v))
print(len(negetive_edges))

66293934


In [77]:
negetive_edges = random.sample(negetive_edges, 5000)
print(len(negetive_edges))
data = []
for i in negetive_edges:
    data.append((i[0],i[1],0))
len(data)

5000


5000

In [78]:
for edge in edges:
    if edge[2] <= 43 and edge[2] >40:
        data.append((edge[0],edge[1],1))
print(len(data))

9618


In [79]:
random.shuffle(data)

In [80]:
from node2vec import Node2Vec

# Generate walks
node2vec = Node2Vec(G, dimensions=100, walk_length=16, num_walks=50)

# train node2vec model
n2w_model = node2vec.fit(window=7, min_count=1)


Generating walks (CPU: 1): 100%|██████████| 50/50 [00:16<00:00,  3.09it/s]


In [91]:
import pandas as pd
# Convert list to DataFrame
# df = pd.DataFrame(data, columns =['source', 'target', 'label'])
n2w_model_dict = {}
for node in G.nodes:
    n2w_model_dict[str(node)] = n2w_model.wv[str(node)]
print(int('9901206') in G.nodes)
result = []
for i, j, _ in data:
    if(i in G.nodes and j in G.nodes):
        result.append(n2w_model_dict[str(i)] + n2w_model_dict[str(j)])

False


In [92]:
print(len(result))

9589


In [95]:
from sklearn.model_selection import train_test_split
link = []
for a,b,i in data:
    if( a in G.nodes and b in G.nodes):
        link.append(i)
xtrain, xtest, ytrain, ytest = train_test_split(np.array(result), link, 
                                                test_size = 0.3, 
                                                random_state = 35)

In [98]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight="balanced")

lr.fit(xtrain, ytrain)

LogisticRegression(class_weight='balanced')

In [99]:
predictions = lr.predict_proba(xtest)

In [114]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
# accuracy_node2vec = accuracy_score(ytest, predictions)
roc_auc_score(ytest, predictions[:,1])


0.7279715464697354